#Danh sách nhóm:
* Nguyễn Công Mỹ Tử - 21522740  
* Nguyễn Văn Đức Huy - 21520930

#Tên bài toán:
Số hóa tủ sách

#Mô tả bài toán:
Input: 1 bức ảnh chụp chính diện, vuông góc với bìa trước của sách, độ phân giải tối thiểu 300x500
Output: tên sách  

Ngữ cảnh ứng dụng: chủ yếu hướng tới những người chưa có danh sách quản lí sách từ đầu (làm việc trong nhà sách cũ hoặc đang có tủ sách tại nhà) muốn quản lí những quyển sách đang có nhưng việc nhập thông tin bằng tay tốn quá nhiều thời gian.

#Mô tả về bộ dữ liệu:
Dữ liệu dùng để train yolov8:
* Ảnh bìa sách được lấy trên Amazon, độ phân giải từ 300x500 trở lên
* File txt chứa các bounding box (title, author, other) tương ứng của mỗi ảnh bìa sách
* Số lượng: 1823 ảnh, 1823 file txt
* train/test = 8/2  

Dữ liệu dùng để test trên openCV:  
* Ảnh bìa sách được lấy trên Amazon, độ phân giải từ 300x500 trở lên, kèm với file txt chứa tên sách tương ứng
* Số lượng: 207 ảnh, 207 file txt

#Mô tả về thuật toán máy học
Trải qua 2 bước:  
* Bước 1: train model yolov8 để detect bounding box của tên sách, đánh giá kết quả bằng mAP50, càng gần 1 detect được bounding box càng tốt.
* Bước 2: dùng opencv để trích xuất tên sách từ bounding box, đánh giá kết quả bằng Levenshtein distance được chuẩn hóa để nằm trong khoảng [0, 100], càng gần 100 hai chuỗi càng giống nhau.


#Cài đặt, tinh chỉnh tham số
* Dùng yolov8n từ thư viện ultralytics
* Tham số được mô hình tự động tìm là SGD(lr=0.01, momentum=0.9)

#Đánh giá kết quả, kết luận
Kết quả khi train yolov8:
* Model sử dụng: yolov8n

* Epoch: 289/1000 (dừng sớm do không cải thiện được trong 50 epoch cuối)
* Batch size: 64

* mAP50 (all class): 0.832
* mAP50 (title): 0.871
* mAP50 (author): 0.76
* mAP50 (other): 0.813  

Kết quả khi test bằng openCV:
* non grayscale average: 45.729468599033815
* grayscale average: 48.009661835748794
* best of 2 average: 51.15942028985507  

Kết luận:
* Do phải trải qua 2 bước, mỗi bước lại có độ chính xác không được cao lắm nên độ chính xác cuối cùng khá thấp.
* Bộ dữ liệu vẫn còn quá ít do hạn chế về nhân lực.







In [ ]:
!pip install ultralytics
!pip install pytesseract
!sudo apt install tesseract-ocr
!pip install python-Levenshtein
!pip install fuzzywuzzy

In [ ]:
from ultralytics import YOLO
from google.colab import drive
import locale
import numpy as np
import pytesseract
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import math
import os
from fuzzywuzzy import fuzz

locale.getpreferredencoding = lambda: "UTF-8"
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [ ]:
drive.mount("/content/gdrive", force_remount=True)

!cp /content/gdrive/MyDrive/hires_dataset_copy.zip /content/
!cp /content/gdrive/MyDrive/data.yaml /content/
!cp /content/gdrive/MyDrive/final_test.zip /content/

!unzip hires_dataset_copy.zip
!unzip final_test.zip

!rm hires_dataset_copy.zip
!rm final_test.zip

In [ ]:
model = YOLO('yolov8n.yaml')

In [ ]:
# train the model
results = model.train(data='data.yaml', epochs=1000, imgsz=640, batch=64)

# consider using this if you don't have time to train
# model = YOLO("epoch289_batch100.pt")

Ultralytics YOLOv8.1.1 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=1000, time=None, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 755k/755k [00:00<00:00, 28.4MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 97.3MB/s]


WARNING ⚠️ NMS time limit 0.550s exceeded
AMP: checks passed ✅


...

   289/1000      8.03G     0.6074     0.4868      0.964        454        640: 100%|██████████| 23/23 [00:29<00:00,  1.28s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:07<00:00,  2.34s/it]

                   all        361       1452      0.816       0.78      0.827      0.668
Stopping training early as no improvement observed in last 50 epochs. Best results observed at epoch 239, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



289 epochs completed in 2.963 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.1 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.87s/it]


                   all        361       1452      0.804      0.777      0.832      0.677
                 title        361        375      0.814      0.824      0.871       0.75
                author        361        366      0.756      0.694       0.76      0.613
                 other        361        711      0.843      0.813      0.863      0.667
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to runs/detect/train


In [ ]:
model = YOLO("/content/runs/detect/train/weights/best.pt")

metrics = model.val()  # no arguments needed, dataset and settings remembered
print(metrics.box.map75)

0.7637893822857807


In [ ]:
!yolo \
task=detect \
mode=predict \
model="/content/epoch50_batch100.pt" \
source="/content/41alKvN9GwL._SL1200_.jpg"

Ultralytics YOLOv8.1.1 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/41alKvN9GwL._SL1200_.jpg: 640x480 1 title, 2 authors, 201.8ms
Speed: 5.3ms preprocess, 201.8ms inference, 786.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!unzip final_test.zip
!rm final_test.zip

In [ ]:
'''
TO-DO:
Increase the size of the corners of the box by 5 pixels,
check if it is valid

store retrieved titles in an array and actual titles in another array
'''
def remove_excessive_spaces(text):
    temp = text.split()
    res = ' '.join(temp)
    return res

def get_coordinates(bounding_box):
    '''
    input: a list: [x of upper left corner,
                    y of upper left corner,
                    x of lower right corner,
                    y of lower right corner]
    output: those numbers, separate, rounded down
    '''
    return math.floor(box[0]), math.floor(box[1]), math.floor(box[2]), math.floor(box[3])

TITLE = 0

image_folder_path = "/content/final_test/images"
title_folder_path = "/content/final_test/titles"

non_graysacle_predicted_titles = []
grayscale_predicted_titles = []
true_titles = []

for image_name in os.listdir(image_folder_path):
    # get image path
    image_path = os.path.join(image_folder_path, image_name)

    # predict on image
    if os.path.isfile(image_path):
        result = model(image_path)

    # get title path
    file_name, file_extension = os.path.splitext(image_name)
    title_path = os.path.join(title_folder_path, file_name+".txt")

    # get title
    if os.path.isfile(title_path):
        with open(title_path, "r", encoding="utf-8") as file:
            title = file.read()
    true_titles.append(title)

    result = result[0]
    classes = result.boxes.cls # classes of bounding boxes
    boxes = result.boxes.xyxy
    coor = []
    #conf = []

    # get title bounding box(es)
    for i in range(len(classes)):
        if classes[i] == TITLE:
            coor.append(boxes[i].cpu().numpy())

            #conf.append(result.boxes.conf[i].item())

    #print(f"coordinates of title: {coor}\n{len(coor)}")
    #image = cv2.imread(res.orig_img)

    non_grayscale_title = ""
    grayscale_title = ""

    for box in coor:
        x1, y1, x2, y2 = get_coordinates(box)

        # not apply grayscale
        title_region = result.orig_img[y1:y2, x1:x2]
        non_grayscale_text = pytesseract.image_to_string(title_region, config='--psm 3')
        non_grayscale_text = remove_excessive_spaces(non_grayscale_text)
        non_grayscale_title += non_grayscale_text + " "

        # apply grayscale
        title_region = cv2.cvtColor(title_region, cv2.COLOR_BGR2GRAY)
        grayscale_text = pytesseract.image_to_string(title_region, config='--psm 3')
        grayscale_text = remove_excessive_spaces(grayscale_text)
        grayscale_title += grayscale_text + " "

        # show image and text in case of checking
        #cv2_imshow(title_region)
        #print(text)
        #print()

    non_graysacle_predicted_titles.append(non_grayscale_title)
    grayscale_predicted_titles.append(grayscale_title)

non_grayscale_ratio = []
grayscale_ratio = []

# get ratio of non grayscale title
for i in range(len(non_graysacle_predicted_titles)):
    non_grayscale_ratio.append(fuzz.ratio(non_graysacle_predicted_titles[i], true_titles[i]))

# get ratio of grayscale title
for i in range(len(grayscale_predicted_titles)):
    grayscale_ratio.append(fuzz.ratio(grayscale_predicted_titles[i], true_titles[i]))

# get best ratio out of 2 above ratio lists
best_ratio = []

for i in range(len(grayscale_ratio)):
    if grayscale_ratio[i] > non_grayscale_ratio[i]:
        ratio = grayscale_ratio[i]
    else:
        ratio = non_grayscale_ratio[i]
    best_ratio.append(ratio)

print(f"\nnon grayscale average: {np.average(non_grayscale_ratio)}")
print(f"grayscale average: {np.average(grayscale_ratio)}")
print(f"best of 2 average: {np.average(best_ratio)}")


image 1/1 /content/final_test/images/test_74.jpg: 640x512 1 title, 1 author, 1 other, 118.7ms
Speed: 4.6ms preprocess, 118.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /content/final_test/images/test_4.jpg: 640x416 1 title, 2 authors, 1 other, 94.8ms
Speed: 2.4ms preprocess, 94.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 416)

image 1/1 /content/final_test/images/test_144.jpg: 640x512 2 titles, 1 author, 2 others, 11.4ms
Speed: 2.8ms preprocess, 11.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /content/final_test/images/test_85.jpg: 640x544 1 title, 1 author, 1 other, 65.1ms
Speed: 2.5ms preprocess, 65.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /content/final_test/images/test_87.jpg: 640x512 1 title, 1 author, 2 others, 10.9ms
Speed: 2.2ms preprocess, 10.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /content/final_test/images/t